In [0]:
# モデルと訓練データを置くGoogleDriveをマウント
from google.colab import drive
drive.mount('/gdrive')

In [0]:
# 訓練データを準備
!unzip -q -d ./ '/gdrive/My Drive/Colab Notebooks/train_data.zip'


In [0]:
# optunaを使う権利をやろう
!pip install optuna

In [0]:
# Formura AI用のデータ読み込みクラス
import numpy as np
from PIL import Image

import math
import random
import os
import collections

from keras.utils import Sequence

frame_offset = 3 # 何フレーム先の状態をラベルに設定するか
class_num = 12

# ファイル名の連番とラベル値で時系列データを作ってるのだが、訳分からなくなってソースがカオス
class F1Data(Sequence):
  def __init__(self, batch_size, data_dir, is_video, frame_num=5, limit=0):
    if not batch_size > frame_num + frame_offset:
      raise ValueError('batch_size must be greater than %d' % (frame_num + frame_offset))
    self.batch_size = batch_size
    self.frame_num = frame_num
    self.data_total_count = len(os.listdir(data_dir))
    self.data_dir = data_dir
    self.is_video = is_video
    self.split_num = math.ceil(self.data_total_count / batch_size) if limit == 0 else limit
    print('data_dir: %s, data_total_count=%s, split_num=%s' % (data_dir, self.data_total_count, self.split_num))
    self.all_file_data = sorted([self.data_tuple(file_name) for file_name in os.listdir(self.data_dir)], key=lambda x: x[0])

  def __getitem__(self, idx):
    return self.make_input_and_label(idx)
  
  def __len__(self):
    return self.split_num
  
  def read_image(self, data_dir, file_name):
    return np.array(Image.open('%s/%s' % (data_dir, file_name)))

  def data_tuple(self, file_name):
    [count, rest] = file_name.split('_')
    state = rest.split('.')[0]
    return (int(count), int(state), file_name)

  def grouped(self, collection, num):
    return zip(*[collection[x:] for x in range(num)])

  def labeled(self, data):
    frames = [x[2] for x in data][:self.frame_num]
    concat_frames = np.stack(frames) if self.is_video else frames[self.frame_num - 1]
    label = data[-1][1]
    one_hot_label = np.identity(class_num)[label]
    return (concat_frames, one_hot_label)

  def make_input_and_label(self, seq_index):
    load_from_idx = (self.batch_size * seq_index) - self.frame_num
    load_from_idx = load_from_idx if load_from_idx > 0 else 0
    image_data = [(x[0], x[1], self.read_image(self.data_dir, x[2])) for x in self.all_file_data[load_from_idx:self.batch_size*(seq_index+1)]]
    grouped_data = self.grouped(image_data, self.frame_num + frame_offset)
    labeled_data = [self.labeled(x) for x in grouped_data]
    [inpus, labels] = zip(*labeled_data)
    return np.stack(inpus), np.stack(labels)
  
  def get_class_weight(self):
    count_data = collections.Counter([x[1] for x in self.all_file_data])
    max_class_count = max(count_data.values())
    return {i: 1 / (count_data[i] / max_class_count) for i in range(class_num)}


In [0]:
# Formura AI用のモデルを作成して保存

import random

import numpy as np

from keras import backend
from keras import applications
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.optimizers import Adam, SGD

import tensorflow as tf

seed = 1
tf.set_random_seed(seed)
np.random.seed(seed)
random.seed(seed)

class_num = 12

# 画像の識別の訓練
def build_cnn_model(): # MobileNet
  model_input = Input(shape=(224, 224, 3), name='input', dtype='float32')
  base_model = applications.MobileNetV2(input_shape=(224, 224, 3), alpha=1.0, pooling='avg', include_top=False, weights='imagenet')
  net = base_model(model_input)
  logits = Dense(class_num, activation='softmax', name='logits')(net)
  model = Model(inputs=model_input, outputs=logits)
  return model

def cnn_train():
  backend.tensorflow_backend.clear_session()

  is_continue = False
  if is_continue:
    model = load_model('/gdrive/My Drive/Colab Notebooks/fai/cnn_model.009-0.778.h5', compile=False)
    initial_epoch = 9
    model.compile(loss='categorical_crossentropy',
#               optimizer=Adam(lr=1e-3, beta_1=0.9, beta_2=0.999),
              optimizer=SGD(lr=5e-5, momentum=0.9, nesterov=True),
              metrics=['accuracy'])
  else:
    model = build_cnn_model()
    initial_epoch = 0
    # SGD(lr=0.00005, momentum=0.9, nesterov=True)
    # Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(lr=0.00005, momentum=0.9, nesterov=True),
                  metrics=['accuracy'])

  model.summary()
  
  epochs = 500
  
  early_stopping = EarlyStopping(monitor='val_loss', patience=25, verbose=1)
  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=1e-15)
  model_cp = ModelCheckpoint('/gdrive/My Drive/Colab Notebooks/fai/cnn_model.{epoch:03d}-{val_loss:.3f}.h5', monitor='val_loss', verbose=0, save_best_only=False, mode='auto', period=1)

  train_data = F1Data(batch_size=60, data_dir='train_data/data_1', is_video=False, limit=0)
  class_weight = train_data.get_class_weight()
  val_data = F1Data(batch_size=60, data_dir='train_data/data_2', is_video=False, limit=0)
  
  hist = model.fit_generator(generator=train_data,
                   epochs=epochs,
                   initial_epoch=initial_epoch,
                   validation_data=val_data,
                   class_weight=class_weight,
                   max_queue_size=10,
                   use_multiprocessing=True, workers=4,
                   callbacks=[early_stopping, reduce_lr, model_cp]
                  )
  
  model.save('/gdrive/My Drive/Colab Notebooks/fai/cnn_model.h5', include_optimizer=False)
  

cnn_train()

# 学習済みモデルの評価
def cnn_evaluate():
  val_inputs1, val_labels1 = F1Data(batch_size=3000, data_dir='train_data/data_2', is_video=False, limit=1)[0]
  val_inputs2, val_labels2 = F1Data(batch_size=3000, data_dir='train_data/data_3', is_video=False, limit=1)[0]
  def evaluate(model_path):
    backend.tensorflow_backend.clear_session()
    model = load_model(model_path)
    print(model_path)
    result = model.evaluate(val_inputs1, val_labels1)
    print(result)
    result = model.evaluate(val_inputs2, val_labels2)
    print(result)
  
  model_dir = '/gdrive/My Drive/Colab Notebooks/fai'
  for file_name in os.listdir(model_dir):
    evaluate('%s/%s' % (model_dir, file_name))
  
cnn_evaluate()


# トレーニング結果
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input (InputLayer)           (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Model) (None, 1280)              2257984   
_________________________________________________________________
logits (Dense)               (None, 12)                15372     
=================================================================
Total params: 2,273,356
Trainable params: 2,239,244
Non-trainable params: 34,112
_________________________________________________________________
data_dir: train_data/data_1, data_total_count=72000, split_num=1200

data_dir: train_data/data_2, data_total_count=28849, split_num=481

Epoch 10/500
1200/1200 [==============================] - 928s 773ms/step - loss: 0.9279 - acc: 0.8070 - val_loss: 0.7905 - val_acc: 0.6796
Epoch 11/500
1200/1200 [==============================] - 923s 769ms/step - loss: 0.8353 - acc: 0.8240 - val_loss: 0.7991 - val_acc: 0.6741
Epoch 12/500
1200/1200 [==============================] - 922s 769ms/step - loss: 0.7519 - acc: 0.8376 - val_loss: 0.7868 - val_acc: 0.6819
Epoch 13/500
1200/1200 [==============================] - 933s 778ms/step - loss: 0.6768 - acc: 0.8522 - val_loss: 0.7989 - val_acc: 0.6850
Epoch 14/500
1200/1200 [==============================] - 933s 777ms/step - loss: 0.6091 - acc: 0.8659 - val_loss: 0.8133 - val_acc: 0.6837
Epoch 15/500
1200/1200 [==============================] - 922s 768ms/step - loss: 0.5479 - acc: 0.8797 - val_loss: 0.8228 - val_acc: 0.6792
Epoch 16/500
1200/1200 [==============================] - 923s 769ms/step - loss: 0.4931 - acc: 0.8919 - val_loss: 0.8293 - val_acc: 0.6865
Epoch 17/500
1200/1200 [==============================] - 924s 770ms/step - loss: 0.4413 - acc: 0.9038 - val_loss: 0.8423 - val_acc: 0.6862

Epoch 00017: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
Epoch 18/500
1200/1200 [==============================] - 924s 770ms/step - loss: 0.3802 - acc: 0.9201 - val_loss: 0.8415 - val_acc: 0.6879
Epoch 19/500
1200/1200 [==============================] - 922s 768ms/step - loss: 0.3698 - acc: 0.9223 - val_loss: 0.8434 - val_acc: 0.6883
Epoch 20/500
1200/1200 [==============================] - 918s 765ms/step - loss: 0.3642 - acc: 0.9237 - val_loss: 0.8460 - val_acc: 0.6873
```

In [0]:
# 後半の部分の訓練のために静止画の特徴量をファイルに保存
import tensorflow as tf
from keras import backend

from keras.models import Model, load_model
import numpy as np

def save_features(save_path, data_dir):
    backend.tensorflow_backend.clear_session()
    
    model = load_model('/gdrive/My Drive/Colab Notebooks/fai2/cnn_model.019-0.843.h5')
    model = model.layers[1]
    
    print(model.inputs)
#     model.summary()
    
    train_data = F1Data(batch_size=1000, data_dir=data_dir, is_video=False)
    
    feature_data = []
    
    for i in range(len(train_data)):
      images, labels = train_data[i]
      features = model.predict(images, verbose=1)
      feature_data += zip(features, labels)
    
    np.save(save_path, feature_data)
    
    
save_features('/gdrive/My Drive/Colab Notebooks/fai/feature_data_1', 'train_data/data_1') # 訓練用
save_features('/gdrive/My Drive/Colab Notebooks/fai/feature_data_2', 'train_data/data_2') # 検証用

In [0]:
# 動画？の学習

import random

import numpy as np

from keras import backend
from keras import applications
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model, load_model
from keras.layers import Input, Dense, Layer, Conv1D, GlobalMaxPooling1D, BatchNormalization, Activation, Lambda, Dropout, LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam, SGD
from keras.utils import Sequence

import tensorflow as tf

seed = 2 # 1
tf.set_random_seed(seed)
np.random.seed(seed)
random.seed(seed)

class_num = 12

# FeatureData
class FeatureData(Sequence):
  def __init__(self, batch_size, data_path, frame_num=15):
    self.batch_size = batch_size
    self.frame_num = frame_num
    self.feature_data = np.load(data_path, allow_pickle=True)
    self.split_num = math.ceil(len(self.feature_data) / batch_size)

  def __getitem__(self, idx):
    batch = self.feature_data[idx*self.batch_size:(idx+1)*self.batch_size+self.frame_num]
    result_frames = []
    result_labels = []
    for j in range(len(batch) - self.frame_num):
      current = batch[j:j+self.frame_num]
      frames, labels = zip(*current)
      result_frames.append(np.array(frames))
      result_labels.append(np.array(labels[-1]))
    return np.array(result_frames), np.array(result_labels)
  
  def __len__(self):
    return self.split_num

# tuning
import optuna

def tuning():
  backend.tensorflow_backend.clear_session()
  
  train_data = F1Data(batch_size=40, data_dir='train_data/data_1', frame_num=18, is_video=True, limit=300)
  class_weight = train_data.get_class_weight()

  train_feature = FeatureData(1000, '/gdrive/My Drive/Colab Notebooks/fai/feature_data_1.npy', frame_num=15)
  val_feature = FeatureData(1000, '/gdrive/My Drive/Colab Notebooks/fai/feature_data_2.npy', frame_num=15)

  
  def objective(trial):
    tf.set_random_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    model_input = Input(shape=(None, 1280), name='input', dtype='float32')

    net = TimeDistributed(Lambda(lambda x: x))(model_input)
    
    conv1_filters = trial.suggest_int('conv1_filters', 128, 384)
    conv1_size = trial.suggest_int('conv1_size', 1, 3)
    conv1_padding = trial.suggest_categorical('conv1_padding', ['same', 'causal'])

    net = Conv1D(conv1_filters, conv1_size, padding=conv1_padding, kernel_initializer='he_normal')(net)
    net = BatchNormalization(axis=2)(net)
    net = Activation('relu')(net)
    net = GlobalMaxPooling1D()(net)

    logits = Dense(class_num, activation='softmax', name='logits')(net)

    model = Model(inputs=model_input, outputs=logits)
    
#     model.summary()
    model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-3, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])

    print(trial.params)
    epochs = 20
    early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
#     prune = optuna.integration.KerasPruningCallback(trial, 'val_loss')
    
    hist = model.fit_generator(
        generator=train_feature,
        epochs=epochs,
        validation_data=val_feature,
        class_weight=class_weight,
        callbacks=[early_stopping],
        verbose=2
      )
    
    best_val = min(hist.history['val_loss'])

    return best_val

  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=150)

  print('best_params: %s' % study.best_params)
  print('best_value: %s' % study.best_value)
  
tuning()
  

# 各フレームの静止画の特徴量を時間軸で畳み込むやつ
def build_1dcnn_model():
  model_input = Input(shape=(None, 1280), name='input', dtype='float32')
  
  net = TimeDistributed(Lambda(lambda x: x))(model_input)
  
  net = Conv1D(226, 3, padding='same', kernel_initializer='he_normal')(net)
  net = BatchNormalization(axis=2)(net)
  net = Activation('relu')(net)
  net = GlobalMaxPooling1D()(net)

  logits = Dense(class_num, activation='softmax', name='logits')(net)
  
  model = Model(inputs=model_input, outputs=logits)
  return model

# train motion
def train_motion():
  backend.tensorflow_backend.clear_session()

  model = build_1dcnn_model()
  initial_epoch = 0
  model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-3, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])

  model.summary()
  
  epochs = 10
  
  model_cp = ModelCheckpoint('/gdrive/My Drive/Colab Notebooks/fai_motion/1dcnn_model.{epoch:03d}-{val_loss:.3f}.h5', monitor='val_loss', verbose=0, save_best_only=False, mode='auto', period=1)

  train_data = F1Data(batch_size=35, data_dir='train_data/data_1', is_video=True, frame_num=15, limit=0)
  class_weight = train_data.get_class_weight()
  train_feature = FeatureData(1000, '/gdrive/My Drive/Colab Notebooks/fai/feature_data_1.npy', frame_num=15)
  val_feature = FeatureData(1000, '/gdrive/My Drive/Colab Notebooks/fai/feature_data_2.npy', frame_num=15)
  
  hist = model.fit_generator(generator=train_feature,
                   epochs=epochs,
                   initial_epoch=initial_epoch,
                   validation_data=val_feature,
                   class_weight=class_weight,
#                    callbacks=[model_cp]
                  )
  

train_motion()

# rnn(ボツ)
def build_cnn_rnn_model(): # MobileNet+LSTM
  model_input = Input(shape=(None, 224, 224, 3), name='input', dtype='float32')
  
  base_model = load_model('/gdrive/My Drive/Colab Notebooks/fai2/cnn_model.019-0.843.h5', compile=False)
  mobilenet = base_model.layers[1]

  net = TimeDistributed(mobilenet)(model_input)
  net = LSTM(320, dropout=0.25, recurrent_dropout=0.25)(net)
  net = Dense(1280, activation='relu', kernel_initializer='he_normal')(net)
  net = Dropout(0.25)(net)

  logits = Dense(class_num, activation='softmax', name='logits')(net)
  
  model = Model(inputs=model_input, outputs=logits)
  model.layers[1].trainable = False
  return model


# optunaさんが出した最適なハイパーパラメータ
best_params: {'conv1_filters': 226, 'conv1_size': 3, 'conv1_padding': 'same'}  
best_value: 0.6584836531015238


# 時間で畳み込んだやつのトレーニング結果
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input (InputLayer)           (None, None, 1280)        0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 1280)        0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 226)         868066    
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 226)         904       
_________________________________________________________________
activation_1 (Activation)    (None, None, 226)         0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 226)               0         
_________________________________________________________________
logits (Dense)               (None, 12)                2724      
=================================================================
Total params: 871,694
Trainable params: 871,242
Non-trainable params: 452
_________________________________________________________________
data_dir: train_data/data_1, data_total_count=72000, split_num=2058
Epoch 1/10
72/72 [==============================] - 11s 146ms/step - loss: 3.4031 - acc: 0.6117 - val_loss: 0.8609 - val_acc: 0.6480
Epoch 2/10
72/72 [==============================] - 10s 141ms/step - loss: 1.6484 - acc: 0.7235 - val_loss: 0.7666 - val_acc: 0.6954
Epoch 3/10
72/72 [==============================] - 9s 132ms/step - loss: 1.3463 - acc: 0.7546 - val_loss: 0.7316 - val_acc: 0.6902
Epoch 4/10
72/72 [==============================] - 9s 129ms/step - loss: 1.1618 - acc: 0.7792 - val_loss: 0.8332 - val_acc: 0.6584
Epoch 5/10
72/72 [==============================] - 9s 127ms/step - loss: 1.0175 - acc: 0.7927 - val_loss: 0.6604 - val_acc: 0.7293
Epoch 6/10
72/72 [==============================] - 10s 133ms/step - loss: 0.9307 - acc: 0.8035 - val_loss: 0.7070 - val_acc: 0.7190
Epoch 7/10
72/72 [==============================] - 9s 128ms/step - loss: 0.8766 - acc: 0.8126 - val_loss: 0.6942 - val_acc: 0.7176
Epoch 8/10
72/72 [==============================] - 9s 131ms/step - loss: 0.8658 - acc: 0.8214 - val_loss: 0.7376 - val_acc: 0.7172
Epoch 9/10
72/72 [==============================] - 10s 132ms/step - loss: 0.7491 - acc: 0.8372 - val_loss: 0.7044 - val_acc: 0.7245
Epoch 10/10
72/72 [==============================] - 10s 138ms/step - loss: 0.6961 - acc: 0.8474 - val_loss: 0.7734 - val_acc: 0.7060

```

In [0]:
# 2つをつなげて動画入力から推論までするモデルを構築して保存
from keras import backend
from keras.models import Model, load_model
from keras.layers import Input
from keras.optimizers import Adam, SGD
from keras.layers.wrappers import TimeDistributed

import tensorflow as tf

def build_fai_model():
  backend.tensorflow_backend.clear_session()
  
  image_cnn_model = load_model('/gdrive/My Drive/Colab Notebooks/fai2/cnn_model.019-0.843.h5', compile=False)
  frame_cnn_model = load_model('/gdrive/My Drive/Colab Notebooks/fai_motion/1dcnn_model.010-0.760.h5', compile=False)
  
  mobilenet = image_cnn_model.layers[1]

  model_input = Input(shape=(None, 224, 224, 3), name='input', dtype='float32')
  net = TimeDistributed(mobilenet)(model_input)
  for layer in frame_cnn_model.layers[2:]:
    net = layer(net)

  model = Model(inputs=model_input, outputs=net)
  return model

# テストデータで性能をテスト
def test():
  model = build_fai_model()
  model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=5e-5, momentum=0.9, nesterov=True),
              metrics=['accuracy'])
  model.summary()
  
  test_data = F1Data(batch_size=50, data_dir='train_data/data_3', frame_num=15, is_video=True)
  test_loss, test_acc = model.evaluate_generator(test_data)
  print('test_loss: %.4f - test_acc: %.4f' % (test_loss, test_acc))
  
test()

def save_model():
  model = build_fai_model()
  model.save('/gdrive/My Drive/Colab Notebooks/fai_motion/fai_model.h5', include_optimizer=False)

save_model()

# テスト結果

```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input (InputLayer)           (None, None, 224, 224, 3) 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 1280)        2257984   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 226)         868066    
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 226)         904       
_________________________________________________________________
activation_1 (Activation)    (None, None, 226)         0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 226)               0         
_________________________________________________________________
logits (Dense)               (None, 12)                2724      
=================================================================
Total params: 3,129,678
Trainable params: 3,095,114
Non-trainable params: 34,564
_________________________________________________________________
data_dir: train_data/data_3, data_total_count=24587, split_num=492
test_loss: 0.6521 - test_acc: 0.7279


```

In [0]:
# Kerasのモデルをtensorflowのsaved_modelに変換して保存
!rm -rf tmp_saved_model

from keras import backend
from keras.models import load_model

import tensorflow as tf

tf.reset_default_graph()
def save_as_tf():
  backend.tensorflow_backend.clear_session()
  
  model = load_model('/gdrive/My Drive/Colab Notebooks/fai_motion/fai_model.h5', compile=False)
  
  session = backend.get_session()
  tf.compat.v1.saved_model.simple_save(
      session,
      './tmp_saved_model',
      inputs={'input': model.input},
      outputs={'output': model.output}
  )
  
save_as_tf()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./tmp_saved_model/saved_model.pb


In [0]:
# モデル確認
!saved_model_cli show --tag_set serve --dir tmp_saved_model
!saved_model_cli show --tag_set serve --signature_def serving_default --dir tmp_saved_model

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "serving_default"
The given SavedModel SignatureDef contains the following input(s):
  inputs['input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, 224, 224, 3)
      name: input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 12)
      name: logits/Softmax:0
Method name is: tensorflow/serving/predict


In [0]:
# いい感じに予測ラベル値が出力になるように、直接tensorflowで弄る
!rm -rf saved_model
import tensorflow as tf

tf.reset_default_graph()

with tf.Session() as session:
  
  tf.compat.v1.saved_model.loader.load(session, ['serve'], 'tmp_saved_model')
  
  # input
  input_frames = session.graph.get_tensor_by_name("input:0")
  # output
  logits = session.graph.get_tensor_by_name("logits/Softmax:0")
  predicted_label = tf.argmax(logits, 1, name='predicted_label')
  

  tf.compat.v1.saved_model.simple_save(session,
                        './saved_model',
                        inputs={'input': input_frames},
                        outputs={'output': predicted_label})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from tmp_saved_model/variables/variables
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_model/saved_model.pb


In [0]:
!zip -r saved_model.zip saved_model

updating: saved_model/ (stored 0%)
updating: saved_model/saved_model.pb (deflated 92%)
updating: saved_model/variables/ (stored 0%)
updating: saved_model/variables/variables.data-00000-of-00001 (deflated 9%)
updating: saved_model/variables/variables.index (deflated 70%)


In [0]:
 !cp saved_model.zip '/gdrive/My Drive/Colab Notebooks/saved_model.zip'